In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [35]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html?page=1&cate_cd=&srt=&searchKey=&keyWord='
url = f'{base_url}{sub_url}'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [36]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [37]:
li = lis[1]

In [38]:
title = li.select_one('.book_tit').get_text().strip()
title

'관리회계 연습문제 풀이집'

In [12]:
book_href = li.select_one('.book_tit > a')['href']
book_href

'/academy/books/book_view.html?p_code=B3261280549'

In [13]:
sub_url = base_url + book_href
sub_url

'https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B3261280549'

In [16]:
sub_res = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [17]:
sub_lis = sub_soup.select('.info_list > li')
len(sub_lis)

7

In [19]:
author = sub_lis[0].get_text().strip()
author

'저자 : James Jiambalvo'

In [20]:
# 없는게 있으므로 처리 필요
translator = sub_lis[1].get_text().strip()
translator

'번역 : 손성진'

In [21]:
date = sub_lis[2].get_text().strip()
date

'출간 : 2023-09-01'

In [23]:
page = sub_lis[3].get_text().strip()
page

'페이지 : 572 쪽'

In [27]:
price = int(sub_soup.select_one('.pbr > strong').string.replace(',', ''))
price

36000

In [25]:
sub_list = []
sub_dict = {}
for sub_li in sub_lis:
    key = sub_li.select_one('strong').get_text().strip()
    value = sub_li.select_one('span').get_text().strip()
    sub_dict[key[:-2]] = value
sub_dict

{'저자': 'James Jiambalvo',
 '번역': '손성진',
 '출간': '2023-09-01',
 '페이지': '572 쪽',
 'ISBN': '9791156646761',
 '물류코드': '4676',
 '': '초급'}

In [41]:
from tqdm import tqdm

In [48]:
base_url = 'https://www.hanbit.co.kr'

data = []
for page in tqdm(range(1, 24)):
    sub_url = f'/academy/books/new_book_list.html?page={page}&cate_cd=&srt=&searchKey=&keyWord='
    url = f'{base_url}{sub_url}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    for li in lis:
        title = li.select_one('.book_tit').get_text().strip()
        price = int(sub_soup.select_one('.pbr > strong').string.replace(',', ''))
        book_href = li.select_one('.book_tit > a')['href']
        sub_url = base_url + book_href
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.info_list > li')
        sub_dict = {}       # sub_lis로 들어가면 sub_dict 초기화 안됨. 여러개가 연속적으로 찍힘.
        # sub_dict이 다 만들어지고 나서 정보를 빼내 와야 함.
        for sub_li in sub_lis:
            key = sub_li.select_one('strong').get_text().strip()
            value = sub_li.select_one('span').get_text().strip()
            sub_dict[key[:-2]] = value
        author = sub_dict['저자']
        translator = sub_dict['번역'] if sub_dict.keys() == '번역' else ' '
        date = sub_dict['출간']
        page = int(sub_dict['페이지'].split()[0])
        book_dict = {'제목': title, '저자' : author, '번역' : translator, '출간일' : date, '페이지' : page, '가격' : price}
        data.append(book_dict)

100%|██████████| 23/23 [02:00<00:00,  5.26s/it]


In [50]:
df = pd.DataFrame(data)
df.head(25)

,제목,저자,번역,출간일,페이지,가격
0,관리회계 7판,James Jiambalvo,,2023-09-01,572,36000
1,관리회계 연습문제 풀이집,손성진,,2023-09-01,272,36000
2,"IT CookBook, 정보보안 이론과 실제(3판)",Mark Stamp,,2023-08-25,464,11700
3,패널데이터와 복합표본을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,,2023-08-18,468,32000
4,"IT CookBook, HTML5 웹 프로그래밍 입문(4판)",윤인성,,2023-08-05,500,34000
5,"STEM CookBook, 경영경제수학 첫걸음","김진한 , 이재원",,2023-07-31,476,29000
6,논리력을 키우는 영어 글쓰기,이요안,,2023-07-28,248,26000
7,"IT CookBook, 난생처음 데이터 분석 with 파이썬","김규석 , 김주희 , 이준",,2023-07-24,512,20000
8,UML다이어그램을 활용한 소프트웨어 창의 설계,"박현석 , 김연정 , 김연우 , 지현진",,2023-07-13,352,26000
9,"IT CookBook, C 언어 자료구조와 알고리즘 for Beginner",우재남,,2023-07-10,536,24000
